<b> This notebook is practice for analyzing data from CESM2 OMIP2 for SOARS Project </b> 

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import xesmf as xe

# the line above is necessary for getting 
# your plot embedded within the notebook
%matplotlib inline

dir = '/chinook/dallmann/SOARS/'
fn_tos = 'tos_Omon_CESM2_omip2_r1i1p1f1_gn_030601-036612_anom.nc'

data_tos = xr.open_dataset(dir+fn_tos)
data_tos

<xarray.Dataset>
Dimensions:    (bnds: 2, nlat: 384, nlon: 320, time: 732, vertices: 4)
Coordinates:
    lon        (nlat, nlon) float64 ...
    lat        (nlat, nlon) float64 ...
  * time       (time) object 0306-01-15 12:00:00 ... 0366-12-15 12:00:00
Dimensions without coordinates: bnds, nlat, nlon, vertices
Data variables:
    lon_bnds   (nlat, nlon, vertices) float64 ...
    lat_bnds   (nlat, nlon, vertices) float64 ...
    time_bnds  (time, bnds) object ...
    tos        (time, nlat, nlon) float32 ...
Attributes: (12/45)
    CDI:                    Climate Data Interface version 1.7.2 (http://mpim...
    history:                Tue Jun 08 14:39:51 2021: cdo ymonsub tos_Omon_CE...
    source:                 CESM2 (2017): atmosphere: CAM6 (0.9x1.25 finite v...
    institution:            National Center for Atmospheric Research
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            OMIP
    ...                     ...
    variable_id:            tos
    variant_info:           CMIP6 omip2 experiment (JRA forcing) with interac...
    variant_label:          r1i1p1f1
    source_type:            OGCM BGC
    comment:                Experiment run for 6 cycles of 1958-2018 JRA55 fo...
    CDO:                    Climate Data Operators version 1.7.2 (http://mpim...

In [2]:
# Change the time to be from DatetimeNoLeap to a Datetime64
data_tos['time'] = np.arange('1958-01-01','2019-01-01',dtype = 'datetime64[M]')
data_tos

<xarray.Dataset>
Dimensions:    (bnds: 2, nlat: 384, nlon: 320, time: 732, vertices: 4)
Coordinates:
    lon        (nlat, nlon) float64 ...
    lat        (nlat, nlon) float64 ...
  * time       (time) datetime64[ns] 1958-01-01 1958-02-01 ... 2018-12-01
Dimensions without coordinates: bnds, nlat, nlon, vertices
Data variables:
    lon_bnds   (nlat, nlon, vertices) float64 ...
    lat_bnds   (nlat, nlon, vertices) float64 ...
    time_bnds  (time, bnds) object 0306-01-01 00:00:00 ... 0367-01-01 00:00:00
    tos        (time, nlat, nlon) float32 ...
Attributes: (12/45)
    CDI:                    Climate Data Interface version 1.7.2 (http://mpim...
    history:                Tue Jun 08 14:39:51 2021: cdo ymonsub tos_Omon_CE...
    source:                 CESM2 (2017): atmosphere: CAM6 (0.9x1.25 finite v...
    institution:            National Center for Atmospheric Research
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            OMIP
    ...                     ...
    variable_id:            tos
    variant_info:           CMIP6 omip2 experiment (JRA forcing) with interac...
    variant_label:          r1i1p1f1
    source_type:            OGCM BGC
    comment:                Experiment run for 6 cycles of 1958-2018 JRA55 fo...
    CDO:                    Climate Data Operators version 1.7.2 (http://mpim...

In [14]:
tos = data_tos['tos']

In [4]:
#regridder = xe.Regridder(tos.mean('time'), data_out, 'bilinear', reuse_weights=False, ignore_degenerate=True)

In [13]:
# Create the target grid (1x1 degree)
ds_out = xe.util.grid_global(1, 1)
ds_out  # contains lat/lon values of cell centers and boundaries.

# Create regridder for `bilinear` interpolation
regridder = xe.Regridder(tos.mean('time'), ds_out, 'bilinear', reuse_weights=True, ignore_degenerate=True)
 
# To reuse weights, you need to provide a filename or weights

# Perform the regridding
dr_out = regridder(tos.mean('time'))

/opt/miniconda3/envs/prod/lib/python3.8/site-packages/xarray/core/dataarray.py:745: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return key in self.data
/opt/miniconda3/envs/prod/lib/python3.8/site-packages/xesmf/backend.py:53: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')


ValueError: To reuse weights, you need to provide either filename or weights.

In [12]:
ds_out

<xarray.Dataset>
Dimensions:  (x: 360, x_b: 361, y: 180, y_b: 181)
Coordinates:
    lon      (y, x) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
    lat      (y, x) float64 -89.5 -89.5 -89.5 -89.5 ... 89.5 89.5 89.5 89.5
    lon_b    (y_b, x_b) int64 -180 -179 -178 -177 -176 ... 176 177 178 179 180
    lat_b    (y_b, x_b) int64 -90 -90 -90 -90 -90 -90 -90 ... 90 90 90 90 90 90
Dimensions without coordinates: x, x_b, y, y_b
Data variables:
    *empty*